In [ ]:
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!pip install kaggle
!kaggle datasets download -d mczielinski/bitcoin-historical-data
!unzip bitcoin-historical-data.zip
!pip install ccxt
!curl -L http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz -O && tar xzvf ta-lib-0.4.0-src.tar.gz
!cd ta-lib && ./configure --prefix=/usr && make && make install && cd - && pip install ta-lib

In [ ]:
import pandas as pd
import talib
import numpy as np

# Load data
data = pd.read_csv('/content/bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv')

# Convert the timestamp to datetime and set as index
data['Timestamp'] = pd.to_datetime(data['Timestamp'], unit='s')
data.set_index('Timestamp', inplace=True)

# Fill any missing values forward then backward
data.fillna(method='ffill', inplace=True)
data.fillna(method='bfill', inplace=True)

# Resample to hourly timeframe for this analysis
data = data.resample('H').agg({
    'Open': 'first',
    'High': 'max',
    'Low': 'min',
    'Close': 'last',
    'Volume_(BTC)': 'sum',
    'Volume_(Currency)': 'sum'
})

# Calculate necessary indicators
data['Next_Close_Change'] = data['Close'].pct_change().shift(-1)
data['SMA50'] = talib.SMA(data['Close'], timeperiod=50)
data['SMA200'] = talib.SMA(data['Close'], timeperiod=200)
data['ATR'] = talib.ATR(data['High'], data['Low'], data['Close'], timeperiod=14)
data['Volume_MA_20'] = data['Volume_(BTC)'].rolling(window=20).mean()
data['Candle_Size'] = data['High'] - data['Low']
data['Prev_High'] = data['High'].shift(1)
data['Prev_Low'] = data['Low'].shift(1)

# Daily resample to calculate daily trends
data_daily = data.resample('D').agg({
    'Open': 'first',
    'High': 'max',
    'Low': 'min',
    'Close': 'last',
    'Volume_(BTC)': 'sum',
    'Volume_(Currency)': 'sum'
})
data_daily['SMA_20'] = talib.SMA(data_daily['Close'], timeperiod=20)
data['Daily_Trend'] = data_daily['SMA_20'].reindex(data.index, method='ffill')

# Trend Identification
data['Trend'] = np.where(data['SMA50'] > data['SMA200'], 'Uptrend', 'Downtrend') # Golden , Death
data['Trend'] = np.where(
    (data['Close'] <= data['SMA50'] * 1.02) & (data['Close'] >= data['SMA50'] * 0.98),
    'Sideways',
    data['Trend']
)


In [ ]:
# Function to calculate bullish or bearish performance periods
def calculate_performance_periods(data, pattern_occurrences, start_period, end_period):
    bullish_counts = []
    bearish_counts = []

    for index in pattern_occurrences.index:
        bull_count = 0
        bear_count = 0

        for i in range(start_period, end_period + 1):
            if index + pd.Timedelta(hours=i) in data.index:
                if data.loc[index + pd.Timedelta(hours=i), 'Close'] > data.loc[index + pd.Timedelta(hours=i-1), 'Close']:
                    bull_count += 1
                elif data.loc[index + pd.Timedelta(hours=i), 'Close'] < data.loc[index + pd.Timedelta(hours=i-1), 'Close']:
                    bear_count += 1

        bullish_counts.append(bull_count)
        bearish_counts.append(bear_count)

    return np.mean(bullish_counts), np.mean(bearish_counts)

# Initialize detailed analysis list
detailed_analysis = []

# Get all candlestick pattern functions from TA-Lib
pattern_functions = [func for func in dir(talib) if func.startswith('CDL')]

# Loop through all candlestick patterns
for pattern in pattern_functions:
    pattern_function = getattr(talib, pattern)
    data[pattern] = pattern_function(data['Open'], data['High'], data['Low'], data['Close'])

    # Identify where the pattern occurs
    pattern_occurrences = data[data[pattern] != 0]

    if not pattern_occurrences.empty:
        # Immediate effects (1-7 hours)
        immediate_bullish, immediate_bearish = calculate_performance_periods(data, pattern_occurrences, start_period=1, end_period=7)

        # Long-term effects (8-24 hours)
        longer_term_bullish, longer_term_bearish = calculate_performance_periods(data, pattern_occurrences, start_period=8, end_period=24)

        # Additional features
        #pattern_occurrences['Volatility'] = pattern_occurrences['ATR']
        pattern_occurrences['Volume_Condition'] = np.where(pattern_occurrences['Volume_(BTC)'] > pattern_occurrences['Volume_MA_20'], 'High Volume', 'Low Volume')
        pattern_occurrences['Candle_Size_Relative'] = pattern_occurrences['Candle_Size'] / pattern_occurrences['ATR']
        pattern_occurrences['Support_Resistance'] = np.where(
            (pattern_occurrences['Close'] > pattern_occurrences['Prev_High']) | (pattern_occurrences['Close'] < pattern_occurrences['Prev_Low']),
            'Breach', 'No Breach'
        )
        pattern_occurrences['Daily_Trend_Alignment'] = np.where(pattern_occurrences['Close'] > pattern_occurrences['Daily_Trend'], 'Uptrend', 'Downtrend')

        # Summarize analysis for this pattern
        analysis_summary = {
            'Pattern': pattern,
            'Occurrences': len(pattern_occurrences),
            'Avg_Next_Close_Change': pattern_occurrences['Next_Close_Change'].mean(),
            'Bullish_Percentage': (pattern_occurrences['Next_Close_Change'] > 0).mean() * 100,
            'Bearish_Percentage': (pattern_occurrences['Next_Close_Change'] < 0).mean() * 100,
            'Avg_Immediate_Bullish_Hours': immediate_bullish,
            'Avg_Immediate_Bearish_Hours': immediate_bearish,
            'Avg_Long_Term_Bullish_Hours': longer_term_bullish,
            'Avg_Long_Term_Bearish_Hours': longer_term_bearish,
            'Uptrend_Percentage': (pattern_occurrences['Trend'] == 'Uptrend').mean() * 100,
            'Downtrend_Percentage': (pattern_occurrences['Trend'] == 'Downtrend').mean() * 100,
            'Sideways_Percentage': (pattern_occurrences['Trend'] == 'Sideways').mean() * 100,
            'High_Volume_Percentage': (pattern_occurrences['Volume_Condition'] == 'High Volume').mean() * 100,
            'Support_Resistance_Breach_Percentage': (pattern_occurrences['Support_Resistance'] == 'Breach').mean() * 100,
            'Daily_Trend_Alignment_Percentage': (pattern_occurrences['Daily_Trend_Alignment'] == pattern_occurrences['Trend']).mean() * 100,
            'Best_Hour_of_Day': pattern_occurrences.groupby(pattern_occurrences.index.hour)['Next_Close_Change'].mean().idxmax()
        }

        # Append the summary to the detailed analysis list
        detailed_analysis.append(analysis_summary)

# Convert the detailed analysis list to a DataFrame for better readability
detailed_analysis_df = pd.DataFrame(detailed_analysis)

# Display the detailed analysis DataFrame
print(detailed_analysis_df)

# Save to CSV if needed
detailed_analysis_df.to_csv('kaggle_hourly_pattern_analysis.csv', index=False)